<a href="https://colab.research.google.com/github/Manish-KT/Suvidha_NGO_ML_Intern-/blob/main/ML_project_text_summ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade transformers
# !pip install --upgrade simplet5

In [ ]:
# getting data from kaggle

! pip install -q kaggle

from google.colab import files
files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list

mkdir: cannot create directory ‘/root/.kaggle’: File exists
ref                                                 title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/global-youtube-statistics-2023     Global YouTube Statistics 2023                  60KB  2023-07-28 15:36:38          12139        405  1.0              
nelgiriyewithana/top-spotify-songs-2023             Most Streamed Spotify Songs 2023                47KB  2023-08-26 11:04:57           1904         76  1.0              
iamsouravbanerjee/airline-dataset                   Airline Dataset                                  4MB  2023-08-30 12:03:12            779         25  1.0              
kacperrabczewski/every-porsche-911                  Every Porsche 911 since 1964     

In [ ]:
# downloading the dataset
!kaggle datasets download -d gowrishankarp/newspaper-text-summarization-cnn-dailymail

newspaper-text-summarization-cnn-dailymail.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# extracting and saving the data
! mkdir training_data
! unzip /content/newspaper-text-summarization-cnn-dailymail.zip -d training_data

mkdir: cannot create directory ‘training_data’: File exists
Archive:  /content/newspaper-text-summarization-cnn-dailymail.zip
replace training_data/cnn_dailymail/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace training_data/cnn_dailymail/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace training_data/cnn_dailymail/validation.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [1]:
# downloading the models

!pip install --quiet pytorch-lightning==1.5.10

!pip install --quiet transformers==4.16.2

In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
from random import sample
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [3]:
# Load dataset and limit training size
train_df = pd.read_csv('/content/training_data/cnn_dailymail/train.csv')
TRAINING_SIZE = 5000
train_df = train_df.iloc[0:TRAINING_SIZE, :].copy()
# Rename columns to match your dataset
train_df.rename(columns={'article': 'source_text', 'highlights': 'target_text'}, inplace=True)
print(train_df.head())

                                         id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   
1  0002095e55fcbd3a2f366d9bf92a95433dc305ef   
2  00027e965c8264c35cc1bc55556db388da82b07f   
3  0002c17436637c4fe1837c935c04de47adb18e9a   
4  0003ad6ef0c37534f80b55b4235108024b407f0b   

                                         source_text  \
0  By . Associated Press . PUBLISHED: . 14:11 EST...   
1  (CNN) -- Ralph Mata was an internal affairs li...   
2  A drunk driver who killed a young woman in a h...   
3  (CNN) -- With a breezy sweep of his pen Presid...   
4  Fleetwood are the only team still to have a 10...   

                                         target_text  
0  Bishop John Folda, of North Dakota, is taking ...  
1  Criminal complaint: Cop used his role to help ...  
2  Craig Eccleston-Todd, 27, had drunk at least t...  
3  Nina dos Santos says Europe must be ready to a...  
4  Fleetwood top of League One after 2-0 win at S...  


In [9]:
# Extract articles and highlights
training_article_ls = list(train_df['source_text'])
training_highlight_ls = list(train_df['target_text'])

In [5]:
# Initialize SimpleT5 model

from simplet5 import SimpleT5
model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [6]:
# Train the model
MAX_EPOCHS = 5
model.train(
    train_df=train_df[0:int(0.7 * TRAINING_SIZE)],
    eval_df=train_df[int(0.7 * TRAINING_SIZE):TRAINING_SIZE],
    source_max_token_len=512,
    target_max_token_len=150,
    batch_size=8,
    max_epochs=MAX_EPOCHS,
    use_gpu=True
)


INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [7]:
# Load the trained model
model_path = ''
rootdir = 'outputs/'
for it in os.scandir(rootdir):
    if it.is_dir() and 'simplet5-epoch-' + str(MAX_EPOCHS - 1) in it.path:
        model_path = it.path
model.load_model("t5", "./" + model_path, use_gpu=True)

In [8]:
# Initialize a T5 model without fine-tuning
no_tune_model = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [11]:
# Generate summaries using different approaches
device = torch.device('cpu')
for index in sample(list(np.arange(len(training_article_ls))), 5):
    original_article = training_article_ls[index]
    original_summary = training_highlight_ls[index]

    print('Original Text:')
    print(original_article)

    print('\n\nSummary Text:')
    print(original_summary)

    print('\n\nFine-tuned Predicted Summary:')
    print(model.predict(original_article))

    print('\n\nPredicted Summary (No Fine-tuning):')
    preprocess_text = original_article.strip().replace("\n", "")
    t5_prepared_text = "summarize: " + preprocess_text
    tokenized_text = tokenizer.encode(t5_prepared_text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                         num_beams=4,
                                         no_repeat_ngram_size=2,
                                         min_length=30,
                                         max_length=150,
                                         early_stopping=True)
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(predicted_summary)
    print('=' * 100, "\n\n")

Token indices sequence length is longer than the specified maximum sequence length for this model (857 > 512). Running this sequence through the model will result in indexing errors


Original Text:
By . Hayley O'keeffe . PUBLISHED: . 05:23 EST, 31 October 2013 . | . UPDATED: . 12:53 EST, 31 October 2013 . When artist Alan Dodd was asked by friends to paint a mural on a blank 50-foot-long wall at their home he transformed it into an optical illusion. Friendly faces behind windows and lifelike archways leading to landscapes are painted onto the cottage, in Eye, Suffolk. And the images give an illusion of a make-believe world within. Nationally-renowned artist Alan Dodd stands with the unusual mural that he painted on the side of a friend's home in Eye, Suffolk . The largest painting in the Eye mural is a blacksmith's forge, which takes in the skyline features of the town's church and castle. Quirky characters and also animals feature on other parts of the house side. More... The picture you never thought you'd see... Anna Wintour eating a burger and fries! Artist gives Vogue editor and her fashion friends a calorie-laden makeover . Rare Italian 'San Gennaro' collecti

Token indices sequence length is longer than the specified maximum sequence length for this model (859 > 512). Running this sequence through the model will result in indexing errors


["Alan Dodd, 71, spent 55 days creating the stunning mural. He says it is an homage to the market town's history and culture."]


Predicted Summary (No Fine-tuning):
artist spent 55 days up a ladder painting the 50-foot-long wall. he says it is an homage to the history of the market town in eye, suffolk - and its history is reflected in the work 'against direct sunlight'


Original Text:
Jailed: Joshua Sadler, 21, was sentenced to 12 months after admitting dangerous driving . The mother of a teenager who was killed when a friend crashed his car just four days after passing his test has spoken of her shock after the driver was jailed for only 12 months. Joshua Sadler, 21, lost control of his silver Renault Clio on a 60mph country road and smashed into a tree, killing front seat passenger Mikey Maguire, 19. Sadler had previously been convicted of a . string of serious motoring offences and twice banned from the road . before he even had a licence. He was seen on the night of the fatal cr

In [12]:
# rouge score
!pip install rouge-score


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24952 sha256=c812fc009ac42d80ca3a7bf01ccd89aec00e14bf45fef35b1430142b80b63f52
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [16]:
from rouge_score import rouge_scorer

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Initialize lists to store ROUGE scores
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

# Loop through your samples and calculate ROUGE scores
for index in sample(list(np.arange(len(training_article_ls))), 5):
    original_article = training_article_ls[index]
    original_summary = training_highlight_ls[index]

    print('Original Text:')
    print(original_article)

    print('\n\nSummary Text:')
    print(original_summary)

    print('\n\nFine-tuned Predicted Summaries:')
    predicted_summaries = model.predict(original_article)

    # Select one of the predicted summaries (choosing the best one based on your criteria)
    predicted_summary_fine_tuned = predicted_summaries[0]
    print(predicted_summary_fine_tuned)

    print('\n\nPredicted Summary (No Fine-tuning):')
    t5_prepared_text = "summarize: " + original_article.strip().replace("\n", "")
    tokenized_text = tokenizer.encode(t5_prepared_text, return_tensors="pt").to(device)
    summary_ids = no_tune_model.generate(tokenized_text,
                                         num_beams=4,
                                         no_repeat_ngram_size=2,
                                         min_length=30,
                                         max_length=150,
                                         early_stopping=True)
    predicted_summary_no_fine_tuning = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(predicted_summary_no_fine_tuning)
    print('=' * 100, "\n\n")

    # Calculate ROUGE scores
    scores = scorer.score(original_summary, predicted_summary_fine_tuned)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

# Calculate average ROUGE scores
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print("Average ROUGE-1 Score:", avg_rouge1)
print("Average ROUGE-2 Score:", avg_rouge2)
print("Average ROUGE-L Score:", avg_rougeL)


Original Text:
By . Craig Hope . Follow @CraigHope_DM . After a 110-year wait, Hull City's European adventure very nearly lasted just seven days until Sone Aluko intervened to prolong their stay. But on the evidence of this lethargic and naïve performance, Steve Bruce's side will have to re-emerge with a newfound vigour and maturity when they face a final qualifier later this month. For now, though, relief is the overriding emotion on Humberside. VIDEO Scroll down for Steve Bruce is delighted for the supporters after a late win . Winner: Sone Aluko celebrates scoring the pivotal goal in the 2-1 win for Hull City against AS Trencin . Technique: The substitute adjusted well to fire home the winner at the KC Stadium . Delight: Hull City's European adventure can now continue . Crucial: Ahmed Elmohamady nods home an equaliser for Hull City against AS Trencin . Dream start: Tomas Malec gave the visitors a shock lead early on in the UEFA Europa League qualifier . HULL CITY:  McGregor 6; Bruce

# Conclusion:

Model appears to perform well in terms of unigram overlap (ROUGE-1), but there is room for improvement in capturing bigram sequences (ROUGE-2) and longer common subsequences (ROUGE-L). Further fine-tuning, experimentation with hyperparameters, or using more advanced summarization techniques may help improve the model's performance in capturing longer and more meaningful phrases in the summaries.

# Some conclusions that we can draw based on these scores:

**ROUGE-1 Score (Rouge-1 F1-Score):** This measures the overlap of unigram (single word) sequences between the generated summaries and the reference summaries. Your model achieves a relatively high ROUGE-1 score of 0.4139, indicating that it captures a good amount of overlapping unigrams with the reference summaries.

**ROUGE-2 Score (Rouge-2 F1-Score):** This measures the overlap of bigram (two-word) sequences between the generated summaries and the reference summaries. Your model achieves a lower ROUGE-2 score of 0.1618, indicating that it captures fewer overlapping bigrams compared to unigrams. This is common in text summarization tasks, as capturing exact sequences of two consecutive words can be challenging.

**ROUGE-L Score (Rouge-L F1-Score):** This metric considers the longest common subsequence between the generated and reference summaries. Your model achieves a moderate ROUGE-L score of 0.2844, indicating that it captures some of the longest common subsequences with the reference summaries.


